In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
document.title = 'Optimization App'
if (document.getElementById('appmode-busy')) {
    window.onbeforeunload = function() {return}
}

In [ ]:
import ipywidgets as ipw
from aiida import load_profile
profile = load_profile()
loading_message = ipw.HTML(value=f"Loaded AiiDA profile {profile.name!r}<br>Hold on to your hats, Optimization app will be here shortly 🚀")
display(loading_message)

In [ ]:
from aiida.plugins import DataFactory
from aiida.orm import StructureData, TrajectoryData
from aiida.orm import load_node

from aiidalab_widgets_base import WizardAppWidget, StructureManagerWidget
from aiidalab_widgets_base import StructureBrowserWidget, SmilesWidget, StructureUploadWidget
from aiidalab_widgets_base.bug_report import install_create_github_issue_exception_handler

In [ ]:
from aiidalab_ispg.app import ISPGWorkChainSelector, TrajectoryDataViewer
from aiidalab_ispg.app import ConformerSmilesWidget
from aiidalab_ispg.app import StructureSelectionStep
from aiidalab_ispg.app.optimization_steps import SubmitOptimizationWorkChainStep, ViewOptimizationStatusAndResultsStep
from aiidalab_ispg.app import ViewAtmospecAppWorkChainStatusAndResultsStep
from aiidalab_ispg.app import static
from aiidalab_ispg import __version__
from aiidalab_ispg.app.widgets import TrajectoryManagerWidget

WORKCHAIN_LABEL = "Conformer Optimization Workflow"

structure_manager_widget = TrajectoryManagerWidget(
    importers=[
        ConformerSmilesWidget(title="SMILES"),
        StructureUploadWidget(title="Upload file", allow_trajectories=True),
        StructureBrowserWidget(title="AiiDA database", query_types=(StructureData, TrajectoryData)),
    ],
    node_class='TrajectoryData',
    viewer=TrajectoryDataViewer(),
    storable=True,
)

structure_selection_description = ipw.Label("Select a structure from one of the following sources and then click \"Confirm\" to go to the next step. ")

structure_selection_step = StructureSelectionStep(
    manager=structure_manager_widget,
    description=structure_selection_description
)
structure_selection_step.auto_advance = True

submit_work_chain_step = SubmitOptimizationWorkChainStep()
submit_work_chain_step.auto_advance = True

view_atmospec_work_chain_status_and_results_step = ViewOptimizationStatusAndResultsStep()
view_atmospec_work_chain_status_and_results_step.auto_advance = True

# Link the application steps
ipw.dlink((structure_selection_step, 'confirmed_structure'), (submit_work_chain_step, 'input_structure'))
ipw.dlink((submit_work_chain_step, 'process'), (view_atmospec_work_chain_status_and_results_step, 'process_uuid'), transform=lambda node: node.uuid if node is not None else None)

# Add the application steps to the application
app = WizardAppWidget(
    steps=[
        ('Select structure', structure_selection_step),
        ('Submit workflow', submit_work_chain_step),
        ('Status & Detailed outputs', view_atmospec_work_chain_status_and_results_step),
])

# Reset all subsequent steps in case that a new structure is selected
def _observe_structure_selection(change):
    with structure_selection_step.hold_sync():
        if structure_selection_step.confirmed_structure is not None and \
                structure_selection_step.confirmed_structure != change['new']:
            app.reset()
structure_selection_step.observe(_observe_structure_selection, 'structure')

# Add process selection header
work_chain_selector = ISPGWorkChainSelector(process_label=WORKCHAIN_LABEL, layout=ipw.Layout(width='auto'))
def _observe_process_selection(change):
    if change['old'] == change['new']:
        return
    pk = change['new']
    if pk is None:
        app.reset()
        app.selected_index = 0
    else:
        process = load_node(pk)
        with structure_manager_widget.hold_sync():
            with structure_selection_step.hold_sync():
                app.selected_index = 2
                structure_manager_widget.input_structure = process.inputs.structure
                structure_selection_step.structure = process.inputs.structure
                structure_selection_step.confirmed_structure = process.inputs.structure
                submit_work_chain_step.process = process

work_chain_selector.observe(_observe_process_selection, 'value')    
ipw.dlink((submit_work_chain_step, 'process'), (work_chain_selector, 'value'),
          transform=lambda node: None if node is None else node.pk)


"""
env = Environment()
template = files(static).joinpath("welcome.jinja").read_text()
style = files(static).joinpath("style.css").read_text()
welcome_message = ipw.HTML(env.from_string(template).render(style=style,staticpath=files(static)))
"""
app_with_work_chain_selector = ipw.VBox(children=[work_chain_selector, app])
footer = ipw.HTML(f'<p style="text-align:right;">Copyright (c) 2022 ISPG team (University of Bristol)&#8195Version: {__version__}</p>')
error_handler_output = ipw.Output()
install_create_github_issue_exception_handler(
    error_handler_output,
    url='https://github.com/ispg-group/aiidalab-ispg/issues/new',
    labels=('bug', 'automated-report'),
)
loading_message.layout.display = "none"
display(error_handler_output, app_with_work_chain_selector, footer)